In [ ]:
import re
from math import log
from functools import total_ordering

@total_ordering
class TQuality:        
    def __init__(self, del_prob, avg_latency, ovh_ratio, identifier=""):
        self.p =   int(del_prob * 20)
        self.l = - int(log(avg_latency) / log(2))
        self.o = - int(ovh_ratio)
        self.name = identifier
    
    # Method for comparing two transmission quality descriptors.
    # 
    # @otherTQ: Transmission quality descriptor to compare with.
    # @return:  Positive, if this object describes better transmission quality;
    #           Negative, if this object describes worse transmission quality;
    #           Zero, if both objects correspond to equivalent transmission quality.
    def compareTo(self, otherTQ):
        if (self.p > otherTQ.p):
            return 1;
        elif (self.p == otherTQ.p and self.l > otherTQ.l):
            return 1;
        elif (self.p == otherTQ.p and self.l == otherTQ.l and self.o > otherTQ.o):
            return 1;
        elif (self.p == otherTQ.p and self.l == otherTQ.l and self.o == otherTQ.o):
            return 0;
        else:
            return -1;
        
    def __gt__(self, otherTQ):
        return self.compareTo(otherTQ) > 0
        
    def __eq__(self, otherTQ):
        return self.compareTo(otherTQ) == 0
    
    def simulation_parameters(self):
        params = re.split("-|_", self.name)
        return params[4] + "|" + params[5] + "|" + params[6] + "|" + params[7]
    
    def quality_metrics(self):
        return str(self.p) + "|" + str(self.l) + "|" + str(self.o)

In [ ]:
from pprint import pprint

a = TQuality(1,1,1)
pprint(vars(a))
print a.compareTo(TQuality(1,0.1,1))

In [ ]:
import glob

rel_file_paths = glob.glob("final_comparison_bus/*.txt")

In [ ]:
import re

sample_path = rel_file_paths[0]

print re.split("-|_", sample_path)

## TODO
* We want to sort the scenarios according to transmission quality.
    * Find a way for reading quality information from the MessageStatsReport's
    * Fill a vector with quality information for all tested scenarios
    * Sort that vector using compareTo

In [ ]:
def getTransmissionQualityFromReport(report_path):
    with open(report_path) as f:
        for line in f:
            split = line.split()
            if "delivery_prob" in line:
                p = float(split[1])
            elif "latency_avg" in line:
                if "NaN" in line:
                    l = 1
                else:
                    l = float(split[1])
            elif "overhead_ratio" in line:
                if "NaN" in line:
                    o = 1
                else:
                    o = float(split[1])

    return TQuality(p, l, o, report_path)

In [ ]:
b = getTransmissionQualityFromReport(sample_path)
pprint(vars(b))

In [ ]:
for report_path in rel_file_paths:
    pprint(vars(getTransmissionQualityFromReport(report_path)))

In [ ]:
TQualityArray = map(getTransmissionQualityFromReport, rel_file_paths)
sortedTQualityArray = sorted(TQualityArray, reverse=True)

In [ ]:
map(lambda q: vars(q), TQualityArray)

In [ ]:
map(lambda q: vars(q), sortedTQualityArray)

In [ ]:
sorted_report_paths = map(lambda q: q.name, sortedTQualityArray)

In [ ]:
result_array = filter(lambda s: "Passive" not in s, map(lambda q: q.simulation_parameters() + "|" + q.quality_metrics(), sortedTQualityArray))

In [ ]:
print "Number of evaluated scenarios: %d" % len(result_array)

for (idx, key) in enumerate(result_array):
    print str(idx) + "|" + key

In [29]:
map(lambda q: q.simulation_parameters() + "|" + q.quality_metrics(), sorted(TQualityArray, key=(lambda q: q.o)))

['EpidemicOracleRouter|128|64M|128k|19|-6|-6180',
 'EpidemicOracleRouter|128|64M|1M|19|-6|-6180',
 'EpidemicRouter|128|1M|1M|2|-8|-6079',
 'EpidemicRouter|128|1M|128k|2|-8|-3504',
 'EpidemicOracleRouter|128|1M|128k|4|-7|-1584',
 'EpidemicOracleRouter|128|1M|1M|4|-7|-1584',
 'EpidemicOracleRouter|64|64M|128k|18|-7|-695',
 'EpidemicOracleRouter|64|64M|1M|18|-7|-695',
 'EpidemicOracleRouter|128|2048M|1M|19|-6|-673',
 'EpidemicOracleRouter|128|2048M|128k|19|-6|-673',
 'EpidemicRouter|128|64M|1M|17|-7|-527',
 'EpidemicRouter|64|1M|1M|2|-7|-520',
 'EpidemicRouter|128|2048M|1M|18|-7|-446',
 'EpidemicRouter|128|2048M|128k|17|-8|-406',
 'EpidemicRouter|128|64M|128k|17|-8|-405',
 'EpidemicOracleRouter|64|2048M|1M|18|-7|-292',
 'EpidemicRouter|64|1M|128k|3|-8|-292',
 'EpidemicOracleRouter|64|2048M|128k|18|-7|-292',
 'EpidemicOracleRouter|64|1M|128k|4|-7|-264',
 'EpidemicOracleRouter|64|1M|1M|4|-7|-264',
 'EpidemicRouter|64|64M|1M|17|-7|-235',
 'EpidemicRouter|64|2048M|1M|18|-7|-206',
 'EpidemicRo

In [ ]:
def print_file(file_path):
    with open(file_path) as f:
        for line in f:
            print line,